<a href="https://colab.research.google.com/github/ithabibi/crawler_google_trends/blob/main/monthlyqueryMCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google trends scraber

## lib import

In [ ]:
!pip install pytrends
# import matplotlib
import pandas as pd
from pytrends.request import TrendReq
import plotly.express as px
import locale
import sys  # for utf8
import datetime
from dateutil.relativedelta import relativedelta
import time

import os.path
from pathlib import Path

## run crawler

In [ ]:
trend1 = "همراه اول" #@param {type:"string"}
trend2 = "ایرانسل" #@param {type:"string"}

start_date = datetime.date(2020, 1, 1)#@param {type:"string"}
end_date = datetime.date(2020, 5, 1)#@param {type:"string"}

appended_data = pd.DataFrame()
# set utf8 for terminal
#sys.stdout.reconfigure(encoding='utf-8')
print("به عنوان کدینگ پیش فرض فعال شد", sys.getdefaultencoding())
# display all rows from dataframe using Pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# add my parameters
trend = TrendReq(hl='fa', tz=330,)
# keyword list
kw_list = [trend1, trend2]

########### while ##############
delta = datetime.timedelta(days=1)

while start_date <= end_date:
  time.sleep(10)
  timeslice = str(start_date) + " " + str(start_date + relativedelta(months=1))
  if os.path.isfile(str(start_date)+".csv"):
    start_date = start_date + relativedelta(months=+1)
    continue
  trend.build_payload(kw_list, cat=0, timeframe=timeslice, geo='', gprop='')
  # create a dataframe of google trends suggestions
  # df = pd.DataFrame(trend.suggestions('همراه اول'))
  # df.head()
  # print(df)
  # print("*" * 55)
  rq = trend.related_queries()
  rq.values()

  #Retrival rising trend1
  df = pd.DataFrame(rq.get(trend1).get('rising'))
  df = df.assign(date=start_date,operator=trend1)
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)

  #Retrival rising trend2
  df = pd.DataFrame(rq.get(trend2).get('rising'))
  df = df.assign(date=start_date,operator=trend2)
  appended_data = [appended_data, df]
  appended_data = pd.concat(appended_data)
  
  appended_data.to_csv(str(start_date) + '.csv')
  print(str(start_date) + " OK")
  appended_data.drop(df.index, inplace=True)
  
appended_data = appended_data.reset_index()
print(appended_data.head(10))
#print(str(pd.read_csv(str(start_date + relativedelta(months=-1)) + ".csv")))

#appended_data.to_csv(timeslice + '.csv')

## merg CSV **file**

In [ ]:
import glob

# list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
print(str(csv_files))

df_csv_append = pd.DataFrame()
 
# append the CSV files
for file in csv_files:
    df = pd.read_csv(file)
    df_csv_append = df_csv_append.append(df, ignore_index=True)
df_csv_append.to_csv('merged_data.csv')
df_csv_append